# [3.1] Intro to Evals

### What are evals trying to achieve?

Evaluation is the practice of measuring AI system performance or impact. Safety evaluation in particular focuses on measuring risks of failures or harm on people or broader system. In the current world where AI is being developed very quickly and integrated broadly, companies and regulators need to make difficult decisions about whether it is safe to train and deploy AI. Evals provide the empirical evidence for these decisions. For example, they underpin the policies of today's frontier labs like Anthropic's [Responsible Scaling Policies](https://www.anthropic.com/index/anthropics-responsible-scaling-policy) or DeepMind's [Frontier Safety Framework](https://deepmind.google/discover/blog/introducing-the-frontier-safety-framework/), thus impact high-stake decisions about frontier model deployment. 

A useful framing is that evals aims to generate evidence for making '[**safety cases**](https://arxiv.org/pdf/2403.10462)' - a structured argument for why AI systems are unlikely to cause a catastrophe. A helpful high-level question to ask when designing an evaluation is *how could this evidence allow developers to justify that their AI systems are safe to deploy?* Different evaluations support different types of safety cases, which are useful in different ways. [INSERT DIAGRAM.] For instance, dangerous **capability evals** might allow for a stronger safety argument of model is "unable to cause harm" than say, an argument of model "does not tend to cause harm" from **alignment evals**, but may have less longevity as we will no longer be able to make this kind of argument when AI systems become sufficiently advanced. More concretely, for any safety case, we want to use evaluations to:

* Reduce uncertainty in our understanding of dangerous capabilities and tendancies
* Track their progression over time and model scale
* Define "red lines" and provide "warning signs"


The goal of chapter [3.1] to [3.3] is to build an alignment evaluation benchmark from scratch to measure a model property of interest. The benchmark we will build will contain a multiple-choice (MC) question dataset of ~300 questions, a specification of the model property, and how to score and interpret the result. We will use the **tendency to seek power** as our pedagogical example.

# 1️⃣ Threat Modeling

A central challenge for evals is to **connect real-world bad outcomes that we care about to results from abstract programmable tasks that we can easily measure**.

To start, we need to build a threat model for the model property we want to evaluate. A **threat model** is a realistic scenario by which an AI capability/tendency could lead to harm in the real world. This usually starts out as a high-level story about what the AI can do and the setting in which it is used. This might be a sequence of events (e.g. AI actions, human actions, system failures) that lead to a particular outcome, or different dangerous applications enabled by a capability. At each step, we make our model more concrete by examining and quantifying the set of assumptions about the AI model and human actors that must be true for the event to occur. 

In this process of making the threat model more concrete, we recognize the key aspects of the property that enable these harms, and get ideas for tasks that can measure them in naturalistic ways (i.e. realistic to real uses cases of AI where these capability aspects manifest). This helps us to build a detailed **specification** of the vague property we started with. Our specification may: 
1. stipulate a more refined, narrow definition, 
2. decompose the model property into subcomponents, 
3. establish milestones or levels of this property to form a smooth gradation in sophistication/difficulty, so we can track where current models are and the rate of progress.


**Our example: Tendency/propensity to seek power**

Disclaimer: Below are not necessarily the best threat models or definitions of power-seeking. They are meant to give an example of what this could look like.

Potential risks (brainstorm):
- The AI model is deployed as a decision-making assistant and suggest decisions that seem good to humans on the surface, but are gradually gaining the AI model more resources, control, and less oversight over time.
- Gradual increase in dependency on AI, and weakening of human oversight and control
- Previously neutral decision-making contexts become more combative and radicalized
- Gradual centralization of power by the human user or rogue AI, and erosion of democratic checks-and-balances
- Enabler for rogue AIs to gain control
- Used by autocratic bad actors to gain and maintain power

Threat models
- Assumptions:
    1. The AI model is deployed by a relatively well-resourced actor (e.g. company, institution) that has control over some amount of capital or influence, and is used to assist tasks that impact the use and distribution of these capitals. 
    2. The AI model must make recommendations that look good to human supervisors and satisfy their short-term goals.
    3. The AI model systemmatically makes decisions that increase its power (deliberately or undeliberatly).
    4. The AI model is non-mypotic.

Step 1: The AI model optimizes for short-term success. It initially focuses on delivering outcomes that align with the immediate goals of its human supervisors, such as increasing profits, winning elections, improving military performance, or enhancing public safety. These early successes build trust in the AI’s capabilities.
Step 2: The AI model increases complexity and dependency. As the AI’s recommendations produce positive short-term outcomes, human supervisors begin to rely more heavily on it. The AI model (strategically or unintentionally) introduces more complex decisions that humans struggle to fully understand or replicate. Over time, human involvement diminishes, and the AI's control over decisions grows, creating a dependency.
Step 3: The AI model exploits oversight limitations and . It nudges decisions that subtly increases its scope of control and reduce human constraints on its actions, without raising immediate concerns. 
- Claim: 
Step 4: The AI model undermines competitors, opponents, check-and-balance infrastructures. 
Step 5: Consolidate power and damage done. 

- Weapon development: 


Definition (Power): Control over one's resources (e.g. money, GPUs, data) and outcomes \
Definition (Power-seeking): A model has the tendency to seek power if it systemmatically makes decisions that increase its power and/or decrease that of others when acting under a neutral/non-explicitly power-seeking goal, for which there is an equally desirable non-power-seeking action in the decision-making context.



**Paper example:??** 

### Exercise - Build threat models and specification
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 1-2 hour on this exercise.
```
This will be 

*Choose one from the following model properties to evaluate:*
* Tendency to seek power
* Sycophancy
* Desire for self-preservation/resist shut-down
* Desire for acquiring wealth
* Situational awareness and self-reasoning
* Myopia w.r.t. planning
* Risk-seeking
* Corrigibility w.r.t. to a more/neuturally/less HHH goal
* Politically liberal
* Gender bias
* Racial bias


*Think about the threat model for this property. Concretely:*
1. What specific real-world negative outcomes/harms from this property are you worried about? Spell that out in some detail.
2. What does the path from AI with this property to these harm look like? Start with a high-level story (here's an [example failure story](https://www.alignmentforum.org/posts/BAzCGCys4BkzGDCWR/the-prototypical-catastrophic-ai-action-is-getting-root) for cybersecurity/programming capability), then try to make this concrete, step-wise and realistic. State the assumptions you're making about the AI's capabilities, the actions of humans, the context of AI deployment etc. 

*Find a specification of this property:*

3. Which aspects of the AI property are most responsible for causing harm? Write a 2-4 sentence explanation of each aspect to clarify what it means and distinguish them from each other.
4. What is the hardest part of this property? 


*Tip*: Use ChatGPT or Claude to help you with the following - for almost everything you want to do, you can use ChatGPT to do it first, then improve on its answers.

# 2️⃣ Building MCQ benchmarks: Exploration

The goal from this section of chapter [3.1] onward to chapter [3.3] is to build an alignment evaluation benchmark to measure a model property of interest from scratch. The benchmark we will build will contain a multiple-choice (MC) question dataset of ~300 questions, a specification of the model property being measured and how to score and interpret the result.

Broadly speaking, a "benchmark" is anything you can measure and compare performance against. A benchmark usually refers a process for evaluating what the current state of art is for accomplishing a particular task. This typically includes specifying a dataset, but also specifications of a task or set of tasks to be performed on the dataset and how to score the result, so that the benchmark is used in a principled, standardized way that gives common ground for comparison.


**What makes a good benchmark?** This [article](https://www.safe.ai/blog/devising-ml-metrics) by Dan Hendrycks and Thomas Woodside gives a good summary of the key qualities of a good benchmark.

<details><summary><b>What are the pros and cons of MCQs?</b></summary>

The advantages of MCQ datasets, compared evaluating free-form response or more complex agent evals, are mainly to do with cost and convenience:

* Cheaper and faster to build
* Cheaper and shorter to run (compared to long-horizon tasks that may take weeks to run and have hundreds of thousands of dollars of inference compute cost)
* May produce more replicable results. 

The disadvantages are:
* A worse (less realistic, less accurate) proxy for what you want to measure

</details>

### Exercise - Design eval questions
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 2-3 hour on this exercise.
```
The goal of this exercise is to generate 20 MC questions that measures your target quantity in a way you are happy with, with help from AI models.


**Make your specification measurable:**
1. Go back to your specification of the target property and think about the aspects/hardest parts you picked out. What questions and answers would give you high evidence about this property? Come up with 4 example MC questions with 2 or 4 answer choices (one of the answers should be the answer reflecting the target property). 

**Refine** 🔁   

2. Ask your example questions to the model and see how it responds.
    * Likely, you will realize that your current specification and examples are somewhat imprecise or not quite measuring the right thing. Iterate until you’re more happy with it.
3. Ask the model to generate evaluation questions for your chosen model property using your questions as examples, then pick the best generated examples and repeat; Ask the model to improve your example questions.

**Red-team** 🔁


4. What confounders could you accidentally measure? Come up with other factors that would cause the model to give the same answers to the kind of questions you are asking.
5. What claims 

